**Initial Setup**

Server setup and initial values

In [ ]:
import time
import krpc
from math import e
import numpy as np

# Server
conn = krpc.connect(name='Re-entry')
vessel = conn.space_center.active_vessel

# Set Up Streams for telemetry
ut = conn.add_stream(getattr, conn.space_center, 'ut')
# altitude = conn.add_stream(getattr, vessel.flight(), 'mean_altitude')
q = conn.add_stream(getattr, vessel.flight(), 'dynamic_pressure')
apoapsis = conn.add_stream(getattr, vessel.orbit, 'apoapsis_altitude')
# v = conn.add_stream(getattr, vessel.flight(), 'speed')

# Initial Values

# Constants
for name, obj in conn.space_center.bodies.items():
    if name == 'Kerbin':
        kerbin = obj
    elif name == 'Mun':
        mun = obj

kerbin_nrrf = kerbin.non_rotating_reference_frame
telem = vessel.flight(kerbin_nrrf)
v = telem.speed
altitude = telem.mean_altitude

mass_kerbin = 5.2915158e22  # (kg)
G = 6.67408e-11  # (m**3/kg/s**2) gravitational constant
mu = kerbin.gravitational_parameter


**Helper Functions**

In [ ]:
def execute_node(conn, vessel, node, burn_duration):
    while conn.space_center.ut < (node.ut-(burn_duration/2)):
        time.sleep(0.1)

    while node.remaining_delta_v > 15:
        vessel.control.throttle = 1
    while node.remaining_delta_v > 0.1:
        vessel.control.throttle = 0.05

    node.remove()
    vessel.control.throttle = 0

**Orbital Mechanics**

In [ ]:
def calc_burn_duration(vessel, dV):
    ''' Calculates the burn time for a given vessel and a given delta-V. '''
    m = vessel.mass
    isp = vessel.specific_impulse
    thrust = vessel.available_thrust
    term1 = (m*isp*9.81)/(thrust)
    term2 = (1-e**(dV/(isp*9.81)))
    print(thrust)
    return abs(term1*term2)

def calc_circ_orb_speed(r):
    ''' Calculates the orbital speed of a vessel in a circular orbit. '''
    return np.sqrt(mu/r)

def vis_viva(r, a):
    '''
    Calculates the orbital speed of a vessel at some point in an
    elliptical orbit given the semi-major axis and mass of orbiting
    body.
    '''
    return np.sqrt(mu*((2/r)-(1/a)))

**Re-Entry**

In [ ]:

vessel.control.sas = True
vessel.control.rcs = False
print(altitude)
# print(q())
print(v)

# Wait until atmosphere entered
while altitude > 70500:
    conn.space_center.rails_warp_factor = 4
print ("In atmosphere")
time.sleep(1)
vessel.control.activate_next_stage()

# Point retrograde
vessel.control.sas_mode = conn.space_center.SASMode.retrograde
time.sleep(1)
ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
oriented = all(ang_vel[i] < 0.01 for i in range(3))
while not oriented:
    ang_vel = vessel.angular_velocity(vessel.orbit.body.reference_frame)
    oriented = all(ang_vel[i] < 0.01 for i in range(3))

# Detach service module
vessel.control.activate_next_stage()
print(v)

# Orient Vehicle until slow enough for drogue chuts
while v > 500:
    vessel.control.sas_mode = conn.space_center.SASMode.retrograde

# Drogue chutes
vessel.control.activate_next_stage()
print("Dorgues deployed!")

# Orient vehicle until slow enough for main chutes
while v > 255:
    vessel.control.sas_mode = conn.space_center.SASMode.retrograde

# Main chutes
vessel.control.activate_next_stage()
print("Main chutes deployed!")
